In [1]:
import spacy
import pandas as pd
from spacy import displacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

nlp = spacy.load('pt')

In [36]:
doc = nlp(r'Quais agentes de doenças podem ser transmitidos aos bovinos pelos carrapatos?')
#[token for token in doc if (token.pos_ == 'NOUN' or token.pos_ =='VERB')]
#displacy.render(doc, style='dep', jupyter=True)
set([(token, token.dep_, token.head.text) for token in doc if (token.pos_ in ['VERB', "NOUN", "ADJ", 'ADV'] and token.dep_ in ['obj', 'nmod', 'amod', 'compound'])])

[(doenças, 'nmod', 'agentes'),
 (pelos, 'amod', 'bovinos'),
 (carrapatos, 'amod', 'bovinos')]

In [3]:
corpus = pd.read_csv('data/Raw/quinhentas_perguntas.csv', sep=',', header=0)
answers = pd.read_csv('data/Raw/quinhentas_respostas.csv', sep=",", header=0)
corpus['resposta'] = answers['resposta']

In [36]:
relations = set()
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
for i, row in corpus.iterrows():
    text = row['pergunta']+" "+row['resposta']
    doc = nlp(text)
    #relations.update((lemmatizer(token.orth_, token.pos_)[0], token.dep_, lemmatizer(token.head.text, token.head.pos_)[0]) for token in doc if (token.pos_ in ['VERB', "NOUN", "ADJ", 'ADV'] and token.dep_ in ['obj', 'nsubj', 'iobj', 'obl', 'amod', 'compound', 'conj', 'flat']))
    for token in doc:
        tLemma = lemmatizer(token.orth_, token.pos_)[0]
        descendants = (w for w in token.children if w.pos_ in ["VERB", "NOUN", "ADJ"])
        for children in descendants:
            cLemma = lemmatizer(children.orth_, children.pos_)[0]
            if children.dep_ in ['obj', 'nsubj', 'iobj']:
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if children.dep_ == 'conj' and children.pos_==token.pos_=='NOUN':
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if token.pos_ == "NOUN" and children.pos_ == "NOUN" and children.dep_ in ['flat', 'compound']:
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if token.pos_ == "NOUN" and children.pos_ == "ADJ" and children.dep_ == "amod":
                relations.add((tLemma, children.dep_, cLemma))
                continue
            if token.pos_ == "NOUN" and children.pos_ == "VERB" and children.dep_ == "obl":
                relations.add((tLemma, children.dep_, cLemma))

            


In [41]:
[w for w in relations if w[1]=='amod']
#usar tfidf

[('vermifugação', 'amod', 'quarta'),
 ('trabalho', 'amod', 'científicos'),
 ('custo', 'amod', 'fáceis'),
 ('nome', 'amod', 'comerciais'),
 ('elemento', 'amod', 'essenciais'),
 ('contagem', 'amod', 'menor'),
 ('clima', 'amod', 'seco'),
 ('estrutura', 'amod', 'leve'),
 ('prejuízo', 'amod', 'econômico'),
 ('perturbaçõe', 'amod', 'fisiológicas'),
 ('sabor', 'amod', 'rançoso'),
 ('teto', 'amod', 'limpos'),
 ('tratamento', 'amod', 'prévio'),
 ('gordura', 'amod', 'corporal'),
 ('necessidade', 'amod', 'alimentares'),
 ('resultado', 'amod', 'econômico'),
 ('resultado', 'amod', 'anteriores'),
 ('controle', 'amod', 'biológico'),
 ('aplicação', 'amod', 'necessária'),
 ('perda', 'amod', 'produtivas'),
 ('substância', 'amod', 'tamponantes'),
 ('contaminação', 'amod', 'biológica'),
 ('porte', 'amod', 'pequeno'),
 ('cio', 'amod', 'menor'),
 ('produção', 'amod', 'vivo'),
 ('dia', 'amod', 'quentes'),
 ('concentração', 'amod', 'maior'),
 ('percentagem', 'amod', 'pequena'),
 ('necessidade', 'amod', 'diári